In [ ]:
import requests
import numpy as np
import itertools

In [ ]:
class Config:
    class Room:
        ID = (f'{i}' for i in itertools.count())
        Types = ['Living Room', 'Kitchen', 'Bedroom', 'Bathroom', 'Balcony', 'Entrance', 'Dining Room', 'Study Room', 'Storage']

    class Edge:
        ID = (f'{i}' for i in  itertools.count())

In [ ]:
def Node(room_type = None, corners = None):
    if room_type is None or room_type not in Config.Room.Types: room_type = np.random.choice(Config.Room.Types)
    if corners is None: corners = int(np.random.choice([4,5,6], p=[1., .0, .0]))
    return {
      "id": next(Config.Room.ID), # Nodes unique id   
      "room_type": room_type, # Room Type in ['Living Room', 'Kitchen', 'Bedroom', 'Bathroom', 'Balcony', 'Entrance', 'Dining Room', 'Study Room', 'Storage', 'Front Door', 'Unknown', 'Interior Door']
      "corners": corners # Number of Corners
    }

In [ ]:
rooms = [Node(*conf) for conf in [(),(),(),(),()]]

edges = []

import igraph as ig

g = ig.Graph.Full(len(rooms))
weights = list(range(g.ecount()))
np.random.shuffle(weights)
g.es["weight"] = weights

for edge in g.spanning_tree(weights=g.es["weight"], return_tree=False):
  a,b = g.es[edge].source, g.es[edge].target
  if a<b:
    edges.append({
      "id": next(Config.Edge.ID), # Edge Unique id
      "source": rooms[a]['id'], # First Nodes id
      "target": rooms[b]['id']  # Second Nodes id
    })

In [ ]:
payload = {
  "nodes": rooms,
  "edges": edges,
  "metrics": False # If True provides the length and width of rooms in pixels
}

In [ ]:
payload

In [ ]:
req = requests.post('http://0.0.0.0:8000/generate', json=payload)

In [ ]:
res = req.json()

In [ ]:
from IPython import display
from base64 import b64decode

svg = res['dataUri'][0]
display.HTML(f'<img src="{svg}" />')

In [ ]:
import io
import PIL.Image
import cairosvg

img = io.BytesIO()
cairosvg.svg2png(url=svg, write_to=img, scale=5)
gray = np.array(PIL.Image.open(img)).mean(axis=-1).astype(int)

occupied = np.argwhere(gray != 255)
top_left = occupied.min(axis=0)
bottom_right = occupied.max(axis=0)
gray = gray[top_left[0]:bottom_right[0]+1, top_left[1]:bottom_right[1]+1]

In [ ]:
from matplotlib import pyplot as plt


plt.imshow(gray>0, cmap='gray')

In [ ]:
floorplan = res['floorplan'][0][0]
floorplan

In [ ]:
import typing
import cv2
import numpy as np

def pairwise(iterable):
    iterator = iter(iterable)
    a = next(iterator, None)
    for b in iterator:
        yield a, b
        a = b

def intify(iterable):
    return [int(x) for x in iterable]

In [ ]:
canvas = np.zeros((floorplan['resolution'], floorplan['resolution']))

lines: typing.Dict[int, typing.List] = {}

for room in floorplan['rooms']:
    if room['category'] in (0,): continue
    for point_from, point_to in pairwise(map(intify, room['corners'] + [room['corners'][0]])):
        angle = np.angle(complex(*np.array(point_to) - np.array(point_from)))
        angle = int(np.trunc(180/np.pi * angle)) % 180
        lines.setdefault(angle, []).append((point_from, point_to))

for i, door in enumerate(floorplan['doors']):
    if door['category'] in (11,13): continue

    corners = np.array(door['corners'])

    diagonal = np.abs(corners[0]-corners[2])
    is_horizontal = diagonal[0] > diagonal[1]
    long_neighbor = 1 if is_horizontal else -1

    angle = np.angle(complex(*corners[long_neighbor] - corners[0]))
    index_angle = int(np.trunc(180/np.pi * angle)) % 180
    target = lines.setdefault(index_angle, [])

    TF = np.array([
        [np.cos(angle), -np.sin(angle)],
        [np.sin(angle), np.cos(angle)]
    ])
    FT = np.flip(TF, (0,1)) # "anti"-transpose for right multiplying

    corners = corners.dot(TF)
    targets = np.array(target).dot(TF)

    long_0, short_0 = np.min([corners[0], corners[2]], axis=0)
    long_1, short_1 = np.max([corners[0], corners[2]], axis=0)
    tolerance = (short_1 - short_0)/2

    crosses_door = np.logical_and.reduce([
        -tolerance <= np.mean(targets[:,:,1], axis=1) - short_0,
        -tolerance <= short_1 - np.mean(targets[:,:,1], axis=1),
        long_0 < np.max(targets[:,:,0], axis=1),
        long_1 > np.min(targets[:,:,0], axis=1)
    ])

    for hit in reversed(np.where(crosses_door)[0]):

        to_split = np.array(target.pop(hit)).dot(TF)
        to_split = np.array(sorted(to_split.tolist(), key=lambda x: x[0]))
        
        short = np.mean(to_split[:,1]) # though short1==short2 is implicitly assumed already

        if (end:=to_split[0][0]) < long_0:
            target.append(
                np.array([
                    np.array([end, short]),
                    np.array([long_0, short])
                ]).dot(FT)
            )
    
        if (end:=to_split[1][0]) > long_1:
            target.append(
                np.array([
                    np.array([long_1, short]),
                    np.array([end, short])
                ]).dot(FT)
            )

            # lines_other.append(((long_0,short), (long_1,short)))

for point_from, point_to in map(lambda x: map(intify, x), itertools.chain(*lines.values())):
    cv2.line(canvas, point_from, point_to, 2, 1)

plt.imshow(np.flip(np.max(canvas)-canvas, axis=0), cmap='gray')

In [ ]:
import shapely

rooms = shapely.GeometryCollection([shapely.LinearRing(room['corners']) for room in floorplan['rooms'] if room['category'] not in (0,)])
doors = shapely.MultiPolygon([shapely.Polygon(door['corners']) for door in floorplan['doors'] if door['category'] not in (11,13)])

shapely.difference(shapely.make_valid(rooms), shapely.make_valid(doors))

In [ ]:
import shapely

lines: typing.Dict[int, typing.Any] = {}

for room in floorplan['rooms']:
    if room['category'] in (0,): continue
    for point_from, point_to in pairwise(map(intify, room['corners'] + [room['corners'][0]])):
        angle = np.angle(complex(*np.array(point_to) - np.array(point_from)))
        angle = int(np.trunc(180/np.pi * angle)) % 180
        lines.setdefault(angle, []).append(shapely.LineString([point_from, point_to]))

for angle, lst in lines.items():
    lines[angle] = shapely.MultiLineString(lst)

for i, door in enumerate(floorplan['doors']):
    if door['category'] in (11,13): continue

    corners = np.array(door['corners'])

    diagonal = np.abs(corners[0]-corners[2])
    is_horizontal = diagonal[0] > diagonal[1]
    long_neighbor = 1 if is_horizontal else -1

    angle = np.angle(complex(*corners[long_neighbor] - corners[0]))
    index_angle = int(np.trunc(180/np.pi * angle)) % 180
    
    lines[index_angle] = lines.get(index_angle, shapely.MultiLineString()).difference(shapely.Polygon(corners))


shapely.GeometryCollection(list(lines.values()))